# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Human Value Detection, Multi-label classification, Transformers, BERT


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Introduction

You are tasked to address the [Human Value Detection challenge](https://aclanthology.org/2022.acl-long.306/).

## Problem definition

Arguments are paired with their conveyed human values.

Arguments are in the form of **premise** $\rightarrow$ **conclusion**.

### Example:

**Premise**: *``fast food should be banned because it is really bad for your health and is costly''*

**Conclusion**: *``We should ban fast food''*

**Stance**: *in favour of*

<center>
    <img src="https://github.com/LorenzoScaioli/NLP_multi-label-text-classification-with-transformers/blob/main/images/human_values.png?raw=1" alt="human values" />
</center>

# [Task 1 - 0.5 points] Corpus

Check the official page of the challenge [here](https://touche.webis.de/semeval23/touche23-web/).

The challenge offers several corpora for evaluation and testing.

You are going to work with the standard training, validation, and test splits.

#### Arguments
* arguments-training.tsv
* arguments-validation.tsv
* arguments-test.tsv

#### Human values
* labels-training.tsv
* labels-validation.tsv
* labels-test.tsv

### Example

#### arguments-*.tsv
```

Argument ID    A01005

Conclusion     We should ban fast food

Stance         in favor of

Premise        fast food should be banned because it is really bad for your health and is costly.
```

#### labels-*.tsv

```
Argument ID                A01005

Self-direction: thought    0
Self-direction: action     0
...
Universalism: objectivity: 0
```

### Splits

The standard splits contain

   * **Train**: 5393 arguments
   * **Validation**: 1896 arguments
   * **Test**: 1576 arguments

### Annotations

In this assignment, you are tasked to address a multi-label classification problem.

You are going to consider **level 3** categories:

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

**How to do that?**

You have to merge (**logical OR**) annotations of level 2 categories belonging to the same level 3 category.

**Pay attention to shared level 2 categories** (e.g., Hedonism). $\rightarrow$ [see Table 1 in the original paper.](https://aclanthology.org/2022.acl-long.306/)

#### Example

```
Self-direction: thought:    0
Self-direction: action:     1
Stimulation:                0
Hedonism:                   1

Openess to change           1
```

### Instructions

* **Download** the specificed training, validation, and test files.
* **Encode** split files into a pandas.DataFrame object.
* For each split, **merge** the arguments and labels dataframes into a single dataframe.
* **Merge** level 2 annotations to level 3 categories.

In [2]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys

# data and numerical management packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier

# useful during debugging (progress bars)
from tqdm import tqdm

In [3]:
!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 780.5 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [4]:
import torch
torch.cuda.is_available()

True

#### Download

In [5]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)


def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")

def clean_download(url, name, folder):
    print(f"Current work directory: {Path.cwd()}")
    dataset_folder = Path.cwd().joinpath("Datasets").joinpath(folder)

    if not dataset_folder.exists():
        dataset_folder.mkdir(parents=True)

    dataset_path = dataset_folder.joinpath(name)

    if not dataset_path.exists():
        download_dataset(dataset_path, url)

In [6]:
arguments_split = ["arguments-training.tsv", "arguments-validation.tsv", "arguments-test.tsv"]
labels_split = ["labels-training.tsv", "labels-validation.tsv", "labels-test.tsv"]

for argument in arguments_split:
    url = "https://zenodo.org/records/8248658/files/" + argument + "?download=1"
    clean_download(url, argument, "Arguments")

for label in labels_split:
    url = "https://zenodo.org/records/8248658/files/" + label + "?download=1"
    clean_download(url, label, "Labels")


Current work directory: /content


arguments-training.tsv?download=1: 1.02MB [00:01, 516kB/s]                           


Download complete!
Current work directory: /content


arguments-validation.tsv?download=1: 369kB [00:01, 276kB/s]                           


Download complete!
Current work directory: /content


...

Download complete!
Current work directory: /content


labels-validation.tsv?download=1: 90.1kB [00:01, 72.3kB/s]                            


Download complete!
Current work directory: /content


labels-test.tsv?download=1: 81.9kB [00:01, 78.5kB/s]                            

Download complete!


#### Encode

In [7]:
# Read the arguments split files into DataFrames
arguments_train_df = pd.read_table('Datasets/Arguments/arguments-training.tsv', sep='\t')
arguments_val_df = pd.read_table('Datasets/Arguments/arguments-validation.tsv', sep='\t')
arguments_test_df = pd.read_table('Datasets/Arguments/arguments-test.tsv', sep='\t')

# Read the labels split files into DataFrames
labels_train_df = pd.read_table('Datasets/Labels/labels-training.tsv', sep='\t')
labels_val_df = pd.read_table('Datasets/Labels/labels-validation.tsv', sep='\t')
labels_test_df = pd.read_table('Datasets/Labels/labels-test.tsv', sep='\t')


#### Merge arguments and labels

In [8]:
train_merge_df = pd.merge(arguments_train_df, labels_train_df, on='Argument ID')
val_merge_df = pd.merge(arguments_val_df, labels_val_df, on='Argument ID')
test_merge_df = pd.merge(arguments_test_df, labels_test_df, on='Argument ID')


#### Merge level 2 annotations to level 3 categories

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

In [9]:
level_2_categories = train_merge_df.columns[4:]

In [10]:
print(type(level_2_categories))

<class 'pandas.core.indexes.base.Index'>


In [11]:
level_2_to_Openness_to_change = level_2_categories[:4]
level_2_to_Self_enhancement = level_2_categories[3:8]
level_2_to_Conservation = level_2_categories[7:14]
level_2_to_Self_transcendence = level_2_categories[13:]

In [12]:
def merge_categories(df):
    df['Openness to change'] = [int(any(df[level_2_to_Openness_to_change].loc[i])) for i in range(len(df))]
    df['Self-enhancement'] = [int(any(df[level_2_to_Self_enhancement].loc[i])) for i in range(len(df))]
    df['Conservation'] = [int(any(df[level_2_to_Conservation].loc[i])) for i in range(len(df))]
    df['Self-transcendence'] = [int(any(df[level_2_to_Self_transcendence].loc[i])) for i in range(len(df))]
    return df.drop(level_2_categories, axis=1)

In [13]:
final_train_df = merge_categories(train_merge_df)
final_val_df = merge_categories(val_merge_df)
final_test_df = merge_categories(test_merge_df)

In [14]:
final_train_df.head()

,Argument ID,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,0,0,1,0
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,0,0,1,0
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,0,1,1,0
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...,0,0,1,1
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...,0,0,1,1


# [Task 2 - 2.0 points] Model definition

You are tasked to define several neural models for multi-label classification.

<center>
    <img src="https://github.com/LorenzoScaioli/NLP_multi-label-text-classification-with-transformers/blob/main/images/model_schema.png?raw=1" alt="model_schema" />
</center>

### Instructions

* **Baseline**: implement a random uniform classifier (an individual classifier per category).
* **Baseline**: implement a majority classifier (an individual classifier per category).

<br/>

* **BERT w/ C**: define a BERT-based classifier that receives an argument **conclusion** as input.
* **BERT w/ CP**: add argument **premise** as an additional input.
* **BERT w/ CPS**: add argument premise-to-conclusion **stance** as an additional input.

### Notes

**Do not mix models**. Each model has its own instructions.

You are **free** to select the BERT-based model card from huggingface.

#### Examples

```
bert-base-uncased
prajjwal1/bert-tiny
distilbert-base-uncased
roberta-base
```

### BERT w/ C

<center>
    <img src="https://github.com/LorenzoScaioli/NLP_multi-label-text-classification-with-transformers/blob/main/images/bert_c.png?raw=1" alt="BERT w/ C" />
</center>

### BERT w/ CP

<center>
    <img src="https://github.com/LorenzoScaioli/NLP_multi-label-text-classification-with-transformers/blob/main/images/bert_cp.png?raw=1" alt="BERT w/ CP" />
</center>

### BERT w/ CPS

<center>
    <img src="https://github.com/LorenzoScaioli/NLP_multi-label-text-classification-with-transformers/blob/main/images/bert_cps.png?raw=1" alt="BERT w/ CPS" />
</center>

### Input concatenation

<center>
    <img src="https://github.com/LorenzoScaioli/NLP_multi-label-text-classification-with-transformers/blob/main/images/input_merging.png?raw=1" alt="Input merging" />
</center>

### Notes

The **stance** input has to be encoded into a numerical format.

You **should** use the same model instance to encode **premise** and **conclusion** inputs.

### Instructions

* **Baseline**: implement a random uniform classifier (an individual classifier per category).
* **Baseline**: implement a majority classifier (an individual classifier per category).

<br/>

* **BERT w/ C**: define a BERT-based classifier that receives an argument **conclusion** as input.
* **BERT w/ CP**: add argument **premise** as an additional input.
* **BERT w/ CPS**: add argument premise-to-conclusion **stance** as an additional input.

#### Text encoding

Transformers typically use [SentencePiece tokenizer](https://github.com/google/sentencepiece) to perform sub-word level tokenization.

In particular, the `transformers` library offers the `AutoTokenizer` class to quickly retrieve our chosen transformer's ad-hoc tokenizer.

The `model_card` variable defines the *path* where to look for our pre-trained model.

You can check [huggingface's hub](https://huggingface.co/models) model hub to pick the model card according to your preference.

In [15]:
from transformers import AutoTokenizer

model_card = 'prajjwal1/bert-tiny'

tokenizer = AutoTokenizer.from_pretrained(model_card)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [16]:
labels = ['Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']
num_labels = len(labels)
id2label = {i:label for i, label in enumerate(labels)}
label2id = {label:i for i, label in enumerate(labels)}
labels

['Openness to change',
 'Self-enhancement',
 'Conservation',
 'Self-transcendence']

Encoding Stance in numerical format

In [17]:
le = LabelEncoder()

def encode_stance(df):
    return le.fit_transform(df['Stance'])

final_train_df['Stance'] = encode_stance(final_train_df)
final_val_df['Stance'] = encode_stance(final_val_df)
final_test_df['Stance'] = encode_stance(final_test_df)

In [18]:
final_train_df

,Argument ID,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A01002,We should ban human cloning,1,we should ban human cloning as it will only ca...,0,0,1,0
1,A01005,We should ban fast food,1,fast food should be banned because it is reall...,0,0,1,0
2,A01006,We should end the use of economic sanctions,0,sometimes economic sanctions are the only thin...,0,1,1,0
3,A01007,We should abolish capital punishment,0,capital punishment is sometimes the only optio...,0,0,1,1
4,A01008,We should ban factory farming,0,factory farming allows for the production of c...,0,0,1,1
...,...,...,...,...,...,...,...,...
5388,E08016,The EU should integrate the armed forces of it...,1,"On the one hand, we have Russia killing countl...",0,1,1,1
5389,E08017,Food whose production has been subsidized with...,1,The subsidies were originally intended to ensu...,0,0,1,1
5390,E08018,Food whose production has been subsidized with...,1,These products come mainly from large enterpri...,0,0,0,1
5391,E08019,Food whose production has been subsidized with...,1,Subsidies often make farmers in recipient coun...,0,0,1,1


In [21]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(final_train_df)
val_dataset = Dataset.from_pandas(final_train_df)
test_dataset = Dataset.from_pandas(final_train_df)

# TODO: create all the other datasets

In [31]:
train_dataset.features.keys()

dict_keys(['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence', '__index_level_0__'])

In [130]:
def preprocess_data_conclusion(dataset):
  # take a batch of texts
  text = dataset["Conclusion"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: dataset[k] for k in dataset.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding



def preprocess_data_conclusion_premise(dataset):
  # take a batch of texts
  text1 = dataset["Conclusion"]
  text2 = dataset["Premise"]
  # encode them
  encoding = tokenizer(text1, text2, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: dataset[k] for k in dataset.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text1), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding



def preprocess_data_conclusion_premise_stance(dataset):
  # take a batch of texts
  text1 = dataset["Conclusion"]
  text2 = dataset["Premise"]
  text3 = list(map(str, dataset["Stance"]))
  text = []
  for i, t in enumerate(text1):
    text.append(t + '[SEP]' + text3[i])
  # encode them
  encoding = tokenizer(text, text2, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: dataset[k] for k in dataset.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text1), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [125]:
encoded_c_train_dataset = train_dataset.map(preprocess_data_conclusion, batched=True, remove_columns=train_dataset.column_names)
encoded_c_val_dataset = val_dataset.map(preprocess_data_conclusion, batched=True, remove_columns=val_dataset.column_names)
encoded_c_test_dataset = test_dataset.map(preprocess_data_conclusion, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

In [92]:
encoded_cp_train_dataset = train_dataset.map(preprocess_data_conclusion_premise, batched=True, remove_columns=train_dataset.column_names)
encoded_cp_val_dataset = val_dataset.map(preprocess_data_conclusion_premise, batched=True, remove_columns=val_dataset.column_names)
encoded_cp_test_dataset = test_dataset.map(preprocess_data_conclusion_premise, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

In [131]:
encoded_cps_train_dataset = train_dataset.map(preprocess_data_conclusion_premise_stance, batched=True, remove_columns=train_dataset.column_names)
encoded_cps_val_dataset = val_dataset.map(preprocess_data_conclusion_premise_stance, batched=True, remove_columns=val_dataset.column_names)
encoded_cps_test_dataset = test_dataset.map(preprocess_data_conclusion_premise_stance, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

### Test to check if everything works

In [132]:
example = encoded_cps_train_dataset[0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [133]:
tokenizer.decode(example['input_ids'])


'[CLS] we should ban human cloning [SEP] 1 [SEP] we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [129]:
example['labels']


[0.0, 0.0, 1.0, 0.0]

In [96]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]


['Conservation']

### Others

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch datasets.

In [134]:
encoded_c_train_dataset.set_format("torch")
encoded_c_val_dataset.set_format("torch")
encoded_c_test_dataset.set_format("torch")

encoded_cp_train_dataset.set_format("torch")
encoded_cp_val_dataset.set_format("torch")
encoded_cp_test_dataset.set_format("torch")

encoded_cps_train_dataset.set_format("torch")
encoded_cps_val_dataset.set_format("torch")
encoded_cps_test_dataset.set_format("torch")

### Implementing a random uniform classifier for each category

In [135]:
otc_uniform_classifier = DummyClassifier(strategy="uniform")
se_uniform_classifier = DummyClassifier(strategy="uniform")
cons_uniform_classifier = DummyClassifier(strategy="uniform")
st_uniform_classifier = DummyClassifier(strategy="uniform")

### Implementing a majority classifier for each category

In [136]:
otc_majority_classifier = DummyClassifier(strategy="prior")
se_majority_classifier = DummyClassifier(strategy="prior")
cons_majority_classifier = DummyClassifier(strategy="prior")
st_majority_classifier = DummyClassifier(strategy="prior")

### Defining a BERT-based classifier that receives an argument **conclusion** as input.

We first need to format input data to be fed as mini-batches in a training/evaluation procedure.<br>
https://discuss.huggingface.co/t/whats-the-input-of-bert/14932

In [137]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [138]:
from transformers import AutoModelForSequenceClassification

c_model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                             problem_type="multi_label_classification",
                                                             num_labels=num_labels,
                                                             id2label=id2label,
                                                             label2id=label2id)

cp_model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                              problem_type="multi_label_classification",
                                                              num_labels=num_labels,
                                                              id2label=id2label,
                                                              label2id=label2id)

cps_model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                               problem_type="multi_label_classification",
                                                               num_labels=num_labels,
                                                               id2label=id2label,
                                                               label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [139]:
print(c_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, element

# [Task 3 - 0.5 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using per-category binary F1-score.
* Compute the average binary F1-score over all categories (macro F1-score).

### Example

You start with individual predictions ($\rightarrow$ samples).

```
Openess to change:    0 0 1 0 1 1 0 ...
Self-enhancement:     1 0 0 0 1 0 1 ...
Conversation:         0 0 0 1 1 0 1 ...
Self-transcendence:   1 1 0 1 0 1 0 ...
```

You compute per-category binary F1-score.

```
Openess to change F1:    0.35
Self-enhancement F1:     0.55
Conversation F1:         0.80
Self-transcendence F1:   0.21
```

You then average per-category scores.
```
Average F1: ~0.48
```

In [140]:
batch_size = 8
metric_name = "f1"

In [141]:
from sklearn.metrics import f1_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

# [Task 4 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate **all** defined models.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Pick **at least** three seeds for robust estimation.
* Compute metrics on the validation set.
* Report **per-category** and **macro** F1-score for comparison.

In [142]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="test_dir",                 # where to save model
    learning_rate=2e-5,
    per_device_train_batch_size=8,         # accelerate defines distributed training
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",           # when to report evaluation metrics/losses
    save_strategy="epoch",                 # when to save checkpoint
    load_best_model_at_end=True,
    report_to='none'                       # disabling wandb (default)
)

In [143]:
from transformers import Trainer

c_trainer = Trainer(
    model=c_model,
    args=args,
    train_dataset=encoded_c_train_dataset,
    eval_dataset=encoded_c_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

cp_trainer = Trainer(
    model=cp_model,
    args=args,
    train_dataset=encoded_cp_train_dataset,
    eval_dataset=encoded_cp_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

cps_trainer = Trainer(
    model=c_model,
    args=args,
    train_dataset=encoded_cps_train_dataset,
    eval_dataset=encoded_cps_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [144]:
c_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.631900,0.614067,0.699906,0.249954


TrainOutput(global_step=675, training_loss=0.6271677992078993, metrics={'train_runtime': 19.665, 'train_samples_per_second': 274.244, 'train_steps_per_second': 34.325, 'total_flos': 6856010969088.0, 'train_loss': 0.6271677992078993, 'epoch': 1.0})

In [145]:
c_trainer.evaluate()

{'eval_loss': 0.6140671372413635,
 'eval_f1': 0.6999062473365721,
 'eval_accuracy': 0.24995364361208974,
 'eval_runtime': 4.0575,
 'eval_samples_per_second': 1329.136,
 'eval_steps_per_second': 166.358,
 'epoch': 1.0}

In [146]:
cp_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.631500,0.612955,0.707274,0.258298


TrainOutput(global_step=675, training_loss=0.6260037231445312, metrics={'train_runtime': 27.52, 'train_samples_per_second': 195.967, 'train_steps_per_second': 24.528, 'total_flos': 6856010969088.0, 'train_loss': 0.6260037231445312, 'epoch': 1.0})

In [147]:
cp_trainer.evaluate()

{'eval_loss': 0.6129547357559204,
 'eval_f1': 0.7072738024837374,
 'eval_accuracy': 0.25829779343593545,
 'eval_runtime': 8.8276,
 'eval_samples_per_second': 610.924,
 'eval_steps_per_second': 76.465,
 'epoch': 1.0}

In [148]:
cps_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.610700,0.607670,0.699906,0.249954


TrainOutput(global_step=675, training_loss=0.6118157958984375, metrics={'train_runtime': 26.619, 'train_samples_per_second': 202.6, 'train_steps_per_second': 25.358, 'total_flos': 6856010969088.0, 'train_loss': 0.6118157958984375, 'epoch': 1.0})

In [149]:
cps_trainer.evaluate()

{'eval_loss': 0.6076701879501343,
 'eval_f1': 0.6999062473365721,
 'eval_accuracy': 0.24995364361208974,
 'eval_runtime': 3.9881,
 'eval_samples_per_second': 1352.283,
 'eval_steps_per_second': 169.255,
 'epoch': 1.0}

In [165]:
text = "I am open to change my mind"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(c_trainer.model.device) for k,v in encoding.items()}

outputs = c_trainer.model(**encoding)

In [154]:
encoding1 = tokenizer(text, return_tensors="pt")
encoding1 = {k: v.to(cp_trainer.model.device) for k,v in encoding.items()}

outputs1 = cp_trainer.model(**encoding)

In [166]:
logits = outputs.logits
logits.shape

torch.Size([1, 4])

In [155]:
logits1 = outputs1.logits
logits.shape

torch.Size([1, 4])

In [167]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Conservation', 'Self-transcendence']


In [156]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits1.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Conservation', 'Self-transcendence']


In [153]:
test_prediction_info = c_trainer.predict(encoded_c_test_dataset)
test_predictions, test_labels = test_prediction_info.predictions, test_prediction_info.label_ids

print(test_predictions.shape)
print(test_labels.shape)

(5393, 4)
(5393, 4)


In [ ]:
compute_metrics(test_prediction_info)

# [Task 5 - 1.0 points] Error Analysis

You are tasked to discuss your results.

### Instructions

* **Compare** classification performance of BERT-based models with respect to baselines.
* Discuss **difference in prediction** between the best performing BERT-based model and its variants.

### Notes

You can check the [original paper](https://aclanthology.org/2022.acl-long.306/) for suggestions on how to perform comparisons (e.g., plots, tables, etc...).

# [Task 6 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Model card

You are **free** to choose the BERT-base model card you like from huggingface.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Model Training

You are **free** to choose training hyper-parameters for BERT-based models (e.g., number of epochs, etc...).

### Neural Libraries

You are **free** to use any library of your choice to address the assignment (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

# The End